# Capstone Project

Dies ist die Analyse der Daten des CDC für Health and Nutrition aus den USA

2013-14: https://www.kaggle.com/datasets/cdc/national-health-and-nutrition-examination-survey \
2017-18: https://www.kaggle.com/datasets/rileyzurrin/national-health-and-nutrition-exam-survey-2017-2018/data

In [5]:
import numpy as np
import pandas as pd

In [3]:
pwd

'c:\\Users\\srnwn\\Documents\\neueFische\\Capstone\\scripts'

In [8]:
data = pd.read_sas("../data/2024/Bodymeasures.xpt")

In [9]:
data

,SEQN,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,...,BMXLEG,BMILEG,BMXARML,BMIARML,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXHIP,BMIHIP
0,130378.0,1.0,86.9,NaN,NaN,NaN,NaN,NaN,179.5,NaN,...,42.8,NaN,42.0,NaN,35.7,NaN,98.3,NaN,102.9,NaN
1,130379.0,1.0,101.8,NaN,NaN,NaN,NaN,NaN,174.2,NaN,...,38.5,NaN,38.7,NaN,33.7,NaN,114.7,NaN,112.4,NaN
2,130380.0,1.0,69.4,NaN,NaN,NaN,NaN,NaN,152.9,NaN,...,38.5,NaN,35.5,NaN,36.3,NaN,93.5,NaN,98.0,NaN
3,130381.0,1.0,34.3,NaN,NaN,NaN,NaN,NaN,120.1,NaN,...,NaN,NaN,25.4,NaN,23.4,NaN,70.4,NaN,NaN,NaN
4,130382.0,3.0,13.6,NaN,NaN,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8855,142306.0,1.0,25.3,NaN,NaN,NaN,NaN,NaN,128.0,NaN,...,32.0,NaN,25.0,NaN,19.0,NaN,57.7,NaN,NaN,NaN
8856,142307.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,143.8,NaN,...,NaN,1.0,34.0,NaN,35.4,NaN,NaN,1.0,NaN,1.0
8857,142308.0,1.0,79.3,NaN,NaN,NaN,NaN,NaN,173.3,NaN,...,41.8,NaN,40.0,NaN,30.6,NaN,98.4,NaN,97.7,NaN
8858,142309.0,1.0,81.9,NaN,NaN,NaN,NaN,NaN,179.1,NaN,...,44.0,NaN,40.0,NaN,30.8,NaN,96.0,NaN,103.3,NaN


In [1]:
import pandas as pd

# Merge duplicate columns. 
def combine_dupes(df: pd.DataFrame):

    # Begin by enumerating duplicates:
    col = df.columns.to_list()

    # Initialize a dictionary to store indices
    index_dict = {}

    # Iterate over the array and store indices for each unique value
    for index, value in enumerate(col):
        if value not in index_dict:
            index_dict[value] = [index]
        else:
            index_dict[value].append(index)

    # Filter out values with only one occurrence (non-duplicates)
    duplicates_dict = {key: value for key, value in index_dict.items() if len(value) > 1}

    # Merge duplicates and enumerate columns that need to be dropped
    col2drop = []
    for _, value in duplicates_dict.items():
        for col in value[1:]:
            df.iloc[:, value[0]] = df.iloc[:, value[0]].combine_first(df.iloc[:, col])
            col2drop.append(col)

    # Drop all but first duplicate columns
    df = df.iloc[:, [j for j, c in enumerate(df.columns) if j not in col2drop]]

    return df

In [3]:
import requests
import logging
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import os
import re


# Modify this line to year of choice. E.g., 2015
BASE_YEAR = "August 2021"

# Modify this to return less information while running (e.g., level=logging.DEBUG)
logging.basicConfig(level=logging.INFO)


class NHANESDataDownloader:
    def __init__(self, base_url, id="SEQN"):
        self.base_url = base_url
        self.id = id
        self.title = None

    def get_soup(self, url):
        response = requests.get(url)
        return BeautifulSoup(response.text, 'html.parser')

    def find_data_urls(self):
        soup = self.get_soup(self.base_url)

        # Find all links, then narrow down to data URLs
        links = soup.find_all('a', href=True)

        # Grab title (years of NHANES data) for file-naming purposes
        self.title = soup.find('h1', href=False).text.strip().replace(" ", "_")

        # Grab datatypes (usually [demographics, dietary, examination, laboratory, questionnaire, limited])
        datatypes = [re.split(r'\n|\t', datatype.text.strip())[-1].split(" ")[0].lower() for datatype in links if datatype['href'].startswith(('../search'))]
        data_urls_list = [urljoin(self.base_url, data_url['href']) for data_url in links if data_url['href'].startswith(('../search'))]
        
        # Zip datatype with list of links. Return result
        data_urls = dict(zip(datatypes, data_urls_list))
        # Delete "limited" file if it exists
        data_urls.pop("limited", "")

        return data_urls

    def extract_and_convert_xpt(self, url, datatype):

        logging.info(f'Now working on {datatype} file')
        
        soup = self.get_soup(url)

        # Find all links, then narrow down to XPT files
        links = soup.find_all('a', href=True)
        xpt_files = [link['href'] for link in links if link['href'].lower().endswith(('.xpt'))]
        dframes = []

        # Download and convert each file
        for xpt in xpt_files:
            logging.info("Extracting " + os.path.basename(xpt))
            try:
                 # Ignore unimportant files
                if self.is_unimportant_file(xpt):
                    logging.info(f"{os.path.basename(xpt)} was skipped because it has duplicate id values or is too large")
                    continue
                # Grab file, and add to queue if passes processing
                xpt_df = self.read_and_process_xpt(url, xpt, datatype)
                dframes.append(xpt_df)

            except Exception as e:
                logging.error(f"Error processing file {os.path.basename(xpt)}: {e}")

        # Clean up data: remove duplicate columns
        df = combine_dupes(pd.concat(dframes, axis=1))
        # Replace values smaller than the threshold=10**-30 with 0
        num = df._get_numeric_data()
        num[num < 10E-30] = 0

        # Drop "SAMPLEID" column, as it was only relevant to pooled data.
        try:
            df.drop("SAMPLEID", axis=1)
        except KeyError:
            pass

        # Extract the file name using the URL and save as CSV
        file_name = os.path.join(self.title, datatype)
        df.to_csv(file_name + ".csv", index=True)

        logging.info(f'Finished and saved {datatype} file')


     # These files are not important or are too large
    def is_unimportant_file(self, xpt):
        lst = ["DR1IFF", "DR2IFF", "DSII", "AUXAR", "PAXHR", "PAXMIN"]
        for l in lst:
            if re.search(l, xpt):
                return True
        return False

    def read_and_process_xpt(self, url, xpt, datatype):
        xpt_url = urljoin(url, xpt)
        xpt_df = pd.read_sas(xpt_url)
        filename = os.path.basename(xpt)

        # For lab data, drop columns ending with "LC". These are comment codes.
        if datatype == "laboratory":
            LC_cols = xpt_df.columns.str.endswith('LC')
            LC_count = LC_cols.sum()
            xpt_df = xpt_df.loc[:, ~LC_cols]
            if LC_count > 0:
                logging.info(f"Skipped {LC_count} variables in {filename} because they are large, unimportant comment codes")
        # For examination data, drop Aux files which are large sensor data.
        if datatype == "examination":
            AUX_cols = xpt_df.columns.str.startswith(("WBX", "TYX"))
            AUX_count = AUX_cols.sum()
            xpt_df = xpt_df.loc[:, ~AUX_cols]
            if AUX_count > 0:
                logging.info(f'Skipped {AUX_count} variables in {filename} because they are large, unhelpful sensor data')

        if self.id not in xpt_df.columns:
            logging.info(f"{filename} skipped because it's not based on individual participants")
            return pd.DataFrame()
        if not xpt_df[self.id].duplicated().any():
            xpt_df.set_index(self.id, inplace=True)
            return xpt_df
        else:
            logging.info(f"{filename} skipped because it has duplicate id values")
            return pd.DataFrame()
        
    def download_data(self):
        # Find all data URLs
        data_urls = self.find_data_urls()

        # Make a folder for the data
        os.makedirs(self.title, exist_ok=True)

        # Download and convert all data directly
        for datatype, data_url in data_urls.items():
            self.extract_and_convert_xpt(data_url, datatype)

def main():
    # Input home page of NHANES data. Default is 2017-2018 data
    base_url = f"https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear={BASE_YEAR}"
    downloader = NHANESDataDownloader(base_url)
    downloader.download_data()

if __name__ == "__main__":
    main()

INFO:root:Now working on demographics file
INFO:root:Extracting DEMO_D.xpt
INFO:root:Extracting DEMO_E.xpt
INFO:root:Extracting DEMO_C.xpt
INFO:root:Extracting DEMO_B.xpt
INFO:root:Extracting DEMO.xpt
INFO:root:Extracting DEMO_F.xpt
INFO:root:Extracting DEMO_G.xpt
INFO:root:Extracting DEMO_H.xpt
INFO:root:Extracting DEMO_I.xpt
INFO:root:Extracting DEMO_J.xpt
INFO:root:Extracting P_DEMO.xpt
INFO:root:Extracting DEMO_L.xpt
INFO:root:Finished and saved demographics file
INFO:root:Now working on dietary file
INFO:root:Extracting DRXIFF.xpt
INFO:root:DRXIFF.xpt skipped because it has duplicate id values
INFO:root:Extracting DRXIFF_B.xpt
INFO:root:DRXIFF_B.xpt skipped because it has duplicate id values
INFO:root:Extracting DR1IFF_C.xpt
INFO:root:DR1IFF_C.xpt was skipped because it has duplicate id values or is too large
INFO:root:Extracting DR1IFF_F.xpt
INFO:root:DR1IFF_F.xpt was skipped because it has duplicate id values or is too large
INFO:root:Extracting DR1IFF_E.xpt
INFO:root:DR1IFF_E.x

KeyboardInterrupt: 

In [14]:
print(os.getcwd())

c:\Users\srnwn\Documents\neueFische\Capstone\scripts
